In [1]:
import cv2
import numpy as np
from collections import deque
from keras.models import load_model
import geocoder
import time
import telepot
import firebase_admin
from firebase_admin import credentials, db

In [2]:
# Initialize Firebase Admin SDK with your credentials
cred = credentials.Certificate("./serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://rvidastech-default-rtdb.firebaseio.com/'
})

In [ ]:
bot = telepot.Bot('6457674371:AAE04wSEIjA4pyEvlYeYgqNQ7H0WgRuTiBg')

def get_current_location():
    # Get current location
    location = geocoder.ip('me')
    location_info = f"LOCATION: {location.city}, {location.state}, {location.country}\n"
    return location_info

def store_data_to_firebase(location_info, timeMoment, video_filename, filename):
    # Get a database reference to a Firebase Realtime Database
    ref = db.reference('violence_alerts')

    # Push data to Firebase Realtime Database
    ref.push({
        'location': location_info,
        'time': timeMoment,
        'video_url': video_filename,
        'image_url': filename
    })

def send_telegram_alert(location_info, timeMoment, video_filename, filename):
    # Send message to Telegram
    bot.sendMessage(-1002037531635, f"VIOLENCE ALERT!!\n{location_info}TIME: {timeMoment}")
    with open(video_filename, 'rb') as video_file:
        bot.sendVideo(-1002037531635, video_file)
    bot.sendPhoto(-1002037531635, photo=open(filename, 'rb'))

def print_results():
    # Initializing variables
    trueCount = 0
    imageSaved = 0
    filename = 'savedImage.jpg'
    video_filename = 'recordedVideo.avi'
    sendAlert = 0

    # Get current location
    location_info = get_current_location()

    # Loading pre-trained Keras model
    print("Loading model ...")
    model = load_model('./modelnew.h5')
    Q = deque(maxlen=128)

    # Capturing video from webcam
    vs = cv2.VideoCapture(0)  # 0 for the default webcam, you may need to change this number if you have multiple webcams
    
    # Check if webcam is opened successfully
    if not vs.isOpened():
        print("Error: Failed to open webcam.")
        return

    # Get webcam dimensions
    W = int(vs.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(vs.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*"MJPG")
    writer = cv2.VideoWriter(video_filename, fourcc, 30, (W, H), True)

    # Looping through video frames
    while True:
        grabbed, frame = vs.read()

        if not grabbed:
            break

        output = frame.copy()

        # Preprocessing frame for model input
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128)).astype("float32")
        frame = frame.reshape(128, 128, 3) / 255

        # Predicting violence probability
        preds = model.predict(np.expand_dims(frame, axis=0))[0]
        Q.append(preds)

        # Calculating average prediction
        results = np.array(Q).mean(axis=0)
        i = (preds > 0.50)[0]  # Thresholding predictions
        label = i

        # Determining text color based on violence label
        text_color = (0, 255, 0)  # default: green
        if label:  # Violence prob
            text_color = (0, 0, 255)  # red
            trueCount += 1

        # Adding text to the output frame
        text = "Violence: {}".format(label)
        FONT = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(output, text, (35, 50), FONT, 1.25, text_color, 3)

        # Writing frame to output video
        writer.write(output)

        # Displaying output frame
        cv2.imshow("Output", output)

        # Saving image and sending alert if violence detected for 50 consecutive frames
        if trueCount == 50:
            if imageSaved == 0:
                if label:
                    cv2.imwrite(filename, output)  # Save the frame
                    imageSaved = 1

            if sendAlert == 0:
                timeMoment = time.strftime("%H:%M:%S \nDate %d-%m-%y")
                store_data_to_firebase(location_info, timeMoment, video_filename, filename)
                send_telegram_alert(location_info, timeMoment, video_filename, filename)
                sendAlert = 1

        # Checking for key press to quit
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

    # Cleaning up
    print("[INFO] cleaning up...")
    if writer is not None:
        writer.release()
    vs.release()
    cv2.destroyAllWindows()


print_results()

Loading model ...


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━